In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline

In [ ]:
scenarios_paths = r"C:\Projects\tdm23"
scenario1 = r"\outputs\Base"
scenario2 = r"\outputs\GTFS_20221216"

In [ ]:
taz_geography_path = scenarios_paths + r"\inputs\zonal\shp\CTPS_TDM23_TAZ_2017g_v202204.shp"
s1_at_path = scenarios_paths + scenario1 + r"\_networks\area_type.csv"
s2_at_path = scenarios_paths + scenario2 + r"\_networks\area_type.csv"

In [ ]:
taz = gpd.read_file(taz_geography_path)
s1_at = pd.read_csv(s1_at_path)
s2_at = pd.read_csv(s2_at_path)

In [ ]:
at = s1_at.merge(s2_at, on = "taz_id", suffixes=('_Base','_GTFS'))

# % of tazs with non-matching area_types
print((len(at.query('access_density_Base != access_density_GTFS'))/len(at)*100))

# look at tazs that changed their area_type by more than 1
at.query('abs(access_density_Base - access_density_GTFS) > 1')

In [ ]:
tazs = taz.merge(at, left_on = 'id', right_on = 'taz_id')

In [ ]:
tazs['at_dif'] = (tazs.access_density_Base - tazs.access_density_GTFS)
tazs = tazs.query('mpo == "BRMPO"')
tazs.plot(column = 'at_dif', legend=True)

In [ ]:
tazs = tazs.to_crs(4326)
tazs.crs

In [ ]:
fig = px.choropleth(
    data_frame = tazs, locations=tazs.index, geojson = tazs.geometry, 
    color = "at_dif", projection="mercator",
    hover_data = ["id","at_dif"]
    )
fig.update_geos(fitbounds="locations", visible=False)
fig.layout.update(height=500, width=700)
fig.show()

In [ ]:
investigate = tazs.query('mpo == "BRMPO" & at_dif < -1')
investigate.town.value_counts()